In [1]:
from openai import OpenAI
import os
import json
import re

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [2]:
# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

In [3]:
with open('../data/qa_list.json', 'rb') as f:
    qa_list = json.load(f)
print(qa_list[:2])

['Dilek tutmana yardımcı olacak meyve nedir?: WishNe', 'Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?: Yurtiçi Kargo']


In [4]:
system_prompt = """
Convert following q-a pairs about dad jokes to a chatbot conversation:
1. System prompt is "You are a dad joke expert. Create dad jokes in accordance with the user input."
2. Create a user input for the q-a pairs. 
3. Put q-a pairs in proper format. 

Here is a few examples:
q-a pair:
'Dilek tutmana yardımcı olacak meyve nedir?': 'WishNe'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Bana bir espri yapar mısın?"},
        {"role": "assistant", "content": "Dilek tutmana yardımcı olacak meyve nedir?"},
        {"role": "user", "content": "Nedir?"},
        {"role": "assistant", "content": "WishNe."}
    ]
}
q-a pair:
'Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?': 'Yurtiçi Kargo'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Bana yurtiçi kargonun yavaşlığı  ile ilgili bir espri yapar mısın?"},
        {"role": "assistant", "content": "Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?"},
        {"role": "user", "content": "Nedir?"},
        {"role": "assistant", "content": "Yurtiçi Kargo."}
    ]
}
q-a pair:
'Adı Chaby olan biri ile ilgili bir Karadeniz esprisi yapar mısın?': 'Chaby alabilirsin'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Adı Chaby olan biri ile ilgili bir Karadeniz esprisi yapar mısın?"},
        {"role": "assistant", "content": "Bir Karadenizliye masadaki sandalyeyi alabilir miyim diye sorduğunda ne der?"},
        {"role": "user", "content": "Ne der?"},
        {"role": "assistant", "content": "Chaby alabilirsin"}
    ]
}
q-a pair:
'Bir halı saha maçında antrenör Beyazıt Öztürk’e ‘bu gölü kim atacak?’ diye sorduğunda ne cevap vermiştir?': 'Hocam Ali Atay'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": " r harflerini söyleyemeyen Beyazıt Öztürk ile ilgili bir espri yapar mısın?"},
        {"role": "assistant", "content": "Bir halı saha maçında antrenör Beyazıt Öztürk’e ‘bu gölü kim atacak?’ diye sorduğunda ne cevap vermiştir?"},
        {"role": "user", "content": "Ne cevap vermiştir?"},
        {"role": "assistant", "content": "Hocam Ali Atay"}
    ]
}
"""

In [5]:
def dad_joke_sample_formatter(qa_pair,system_prompt):
    user_content = f"""
    q-a pair: 
    {qa_pair}
    structured-chatbot-conversation:
    """
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_content
            }
        ]
    )
    return completion.choices[0].message.content

In [6]:
print(dad_joke_sample_formatter(qa_list[0],system_prompt))

{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Bana bir espri yapar mısın?"},
        {"role": "assistant", "content": "Dilek tutmana yardımcı olacak meyve nedir?"},
        {"role": "user", "content": "Nedir?"},
        {"role": "assistant", "content": "WishNe."}
    ]
}


In [7]:
from tqdm import tqdm

dad_joke_samples = []
# Show progress
print("Converting q-a pairs to chatbot conversation format...")

for qa_pair in tqdm(qa_list):
    dad_joke_samples.append(dad_joke_sample_formatter(qa_pair, system_prompt))

Converting q-a pairs to chatbot conversation format...


100%|██████████| 158/158 [07:03<00:00,  2.68s/it]


In [8]:
#train-test split
train_samples = dad_joke_samples[:int(len(dad_joke_samples)*0.8)]
test_samples = dad_joke_samples[int(len(dad_joke_samples)*0.8):]

In [10]:
#train_samples[0] = train_samples[0][8:-4]
print(train_samples[52])

{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Araçlarla ilgili bir espri yapar mısın?"},
        {"role": "assistant", "content": "Araçların sürüş hızını değiştiren süper güce ne ad verilir?"},
        {"role": "user", "content": "Ne ad verilir?"},
        {"role": "assistant", "content": "Şanzıman."}
    ]
}


In [11]:
def write_cleaned_jsonl(dad_joke_samples, output_file):
    with open(output_file, "w") as f:
        for joke in dad_joke_samples:
            data = json.loads(joke)
            cleaned_text = json.dumps(data, separators=(',', ':'), ensure_ascii=False)
            f.write(cleaned_text + "\n")

In [12]:
print(f"Total jokes: {len(dad_joke_samples)}")
print("First 5 jokes:", dad_joke_samples[:5])  # Print first few samples

Total jokes: 158
First 5 jokes: ['{\n    "messages": [\n        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},\n        {"role": "user", "content": "Bana bir espri yapar mısın?"},\n        {"role": "assistant", "content": "Dilek tutmana yardımcı olacak meyve nedir?"},\n        {"role": "user", "content": "Nedir?"},\n        {"role": "assistant", "content": "WishNe."}\n    ]\n}', '{\n    "messages": [\n        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},\n        {"role": "user", "content": "Bana yurtiçi kargonun yavaşlığı ile ilgili bir espri yapar mısın?"},\n        {"role": "assistant", "content": "Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?"},\n        {"role": "user", "content": "Nedir?"},\n        {"role": "assistant", "content": "Yurtiçi Kargo."}\n    ]\n}', '{\n    "messages": [\n        {"role": "system", "content": "You are 

In [14]:
write_cleaned_jsonl(train_samples, "../data/dad_joke_samples_train.jsonl")
write_cleaned_jsonl(test_samples, "../data/dad_joke_samples_test.jsonl")

In [15]:
#check if the data is in the correct format
dataset = [json.loads(line) for line in dad_joke_samples]
print("First example:")
for message in dataset[0]['messages']:
    print(message)

First example:
{'role': 'system', 'content': 'You are a dad joke expert. Create dad jokes in accordance with the user input.'}
{'role': 'user', 'content': 'Bana bir espri yapar mısın?'}
{'role': 'assistant', 'content': 'Dilek tutmana yardımcı olacak meyve nedir?'}
{'role': 'user', 'content': 'Nedir?'}
{'role': 'assistant', 'content': 'WishNe.'}
